In [330]:
from freud import box, environment
%matplotlib notebook
from matplotlib import pyplot
import hoomd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [302]:
def thermalize(points, max_dist):
    random_vecs=np.random.rand(len(points))
    return np.array([np.add(np.multiply(np.random.uniform(-max_dist,max_dist),[r,(1.0-r**2.0)**0.5,0]),p) for r,p in zip(random_vecs,points)])

In [265]:
def viz_struct(ax,my_ref_points):
    ax.scatter(my_ref_points[:,0],my_ref_points[:,1], color='k')
    ax.set_aspect('equal')
    ax.set_title('Initial Structure')
    ax.axis('off')

In [270]:
def viz_environment(ax,my_env, my_ref_points):
    for i in set(my_env.getClusters()):
        ax.scatter(my_ref_points[my_env.getClusters()==i][:,0],my_ref_points[my_env.getClusters()==i][:,1])
    ax.set_aspect('equal')
    ax.set_title('With Environments')
    ax.axis('off')

In [271]:
def run_struct(ref_points, therm, threshold, m_e):
    fig, ax= pyplot.subplots(1,2, figsize=(8,4))
    viz_struct(ax[0],ref_points)
    print()
    m_e.cluster(points=ref_points, threshold=threshold)
    viz_environment(ax[1],m_e, ref_points)
    fig.canvas.draw()

In [358]:
def run_sc(therm=0.2, threshold=0.5):
    a=1.0
    hoomd.context.initialize('');
    uc = hoomd.lattice.unitcell(N=1,
                                a1=[1, 0, 0],
                                a2=[0, 1, 0],
                                a3=[0, 0, 1],
                                dimensions=2,
                                position=[[0,0,0]],
                                type_name=['A']);
    snap = uc.get_snapshot();
    snap.replicate(10,10,1);
    ref_points=snap.particles.position
    bbox = box.Box.from_box(snap.box)
    ref_points=thermalize(ref_points, therm)
    bbox.wrap(ref_points)
    me=environment.MatchEnv(bbox, a*0.5, 8)
    run_struct(ref_points, therm, threshold,me)

In [364]:
def run_scs(therm=0.2, threshold=0.5):
    a=1.0
    hoomd.context.initialize('');
    uc = hoomd.lattice.unitcell(N=3,
                                a1=[1, 0, 0],
                                a2=[0, 1, 0],
                                a3=[0, 0, 1],
                                dimensions=2,
                                position=[[0,0,0],[0,0.5,0],[0.5,0.0,0]],
                                type_name=['A','A','A']);
    snap = uc.get_snapshot();
    snap.replicate(10,10,1);
    ref_points=snap.particles.position
    bbox = box.Box.from_box(snap.box)
    
    ref_points=thermalize(ref_points, therm)
    bbox.wrap(ref_points)
    me=environment.MatchEnv(bbox, a*0.5, 8)
    run_struct(ref_points, therm, threshold,me)

In [365]:
interact(run_scs, therm=(0.0,1.0,0.05), threshold=(0.,2.0,0.05));

In [367]:
def run_tri(therm=0.2, threshold=0.5):
    a=1.0
    hoomd.context.initialize('');
    uc = hoomd.lattice.unitcell(N=1,
                                a1=[1, 0, 0],
                                a2=[np.cos(np.pi/6.), np.sin(np.pi/6.), 0],
                                a3=[0, 0, 1],
                                dimensions=2,
                                position=[[0,0,0]],
                                type_name=['A']);
    snap = uc.get_snapshot();
    snap.replicate(10,10,1);
    ref_points=snap.particles.position
    bbox = box.Box.from_box(snap.box)
    
    ref_points=thermalize(ref_points, therm)
    bbox.wrap(ref_points)
    me=environment.MatchEnv(bbox, a*0.5, 8)
    run_struct(ref_points, therm, threshold,me)

In [369]:
interact(run_tri, therm=(0.0,1.0,0.05), threshold=(0.,2.0,0.05));